In [3]:
import pandas as pd

class FeatureRecorder():
    def __init__(self, recorder_config) -> None:
        self.logs = []
        self.records = {}
        self.length = 0
        self.config = recorder_config
        for config_item in recorder_config:
            self.records[config_item] = []

    def begin_recording(self,  remain_content=True, remain_record_objective=True):
        if not remain_record_objective:
            self.clear_all()
        elif not remain_content:
            self.clear_all_content()
        self.logs.append([self.length, -1])
        
    
    def end_recording(self):
        self.logs[-1][1] = self.length - 1
            
    def record_at_t(self, window, time = -1):
        record_name = 0
        for record_name in self.records:
            item = self.config[record_name]
            if not item['enabled']:
                val = None
            else:
                func = item['func']
                val = func(window)
            if time < 0 or time >= self.length:
                self.records[record_name].append(val)
            else:
                self.records[record_name][time] = val
        self.length += 1
            
    def append_record(self, window = -1):
        self.record_at_t(window, -1)
        
    def clear_all_content(self):
        return
    
    def clear_all():
        return
    def get_dataframe(self):
        return pd.DataFrame(self.records)
    
        
    
class RecordConfiguration():
    def __init__(self) -> None:
        self.record_objectives = {
        }
    def add_record_object(self, name, func):
        self.record_objectives[name] = {'enabled': True, 'func': func}

    def enable_record_item(self, name):
        if name not in self.record_objectives:
            return
        self.record_objectives[name]['enabled'] = True
        
    def __iter__(self):
        return self.record_objectives.__iter__()
    
    def __getitem__(self, key):
        return self.record_objectives[key]
        
class FeatureExtractor():
    def __init__(self, freq, decision_time_delay):
        self.freq = freq
        self.decision_time_delay = decision_time_delay
        
    
    def do_extraction(self, signal, feature_recorder, append_record_mode = True, time_limit = -1):
        current_time = 0
        def window_views(signal, time_delay):
            T = signal.shape[0]
            for t in range(0, T):
                yield signal[max(0, t - time_delay): t + 1, :]
        
        if append_record_mode:
            feature_recorder.begin_recording(True, True)
        else:
            feature_recorder.begin_recording(False, True)

            
            
        for window in window_views(signal, int(self.freq * self.decision_time_delay)):
            if append_record_mode:
                feature_recorder.append_record(window)
            else:
                feature_recorder.record_at_t(window, current_time)
            current_time += 1
            if time_limit >= 0 and current_time >= time_limit:
                break
            
        feature_recorder.end_recording()
        return feature_recorder

In [4]:
import numpy as np
import os, sys

In [5]:
sys.path.append("../data/dataset")
sys.path.append("..//microstate_lib/code")
from dataset import *

In [6]:
dataset_facade = EEGDatasetFacade(dataset_base_path="../data")

In [7]:
dataset = dataset_facade("ethz-ieeg")

In [8]:
data = dataset.get_mne_data(['long', 1, 118])

Creating RawArray with float64 data, n_channels=88, n_times=1843200
    Range : 0 ... 1843199 =      0.000 ...  3599.998 secs
Ready.


In [9]:
freq = data.info['sfreq']
feature_extractor = FeatureExtractor(freq, int(10))

In [10]:
import scipy
config = RecordConfiguration()
n_channels = 88
def flatten_remove_diag(x):
    x_no_diag = np.ndarray.flatten(x)
    x_no_diag = np.delete(x_no_diag, range(0, len(x_no_diag), len(x) + 1), 0)
    x_no_diag = x_no_diag.reshape(len(x), len(x) - 1)
    return x_no_diag.ravel()

def calculate_auto_corr(signal, lag):
    corr = 0
    for t in range(lag, signal.shape[0]):
        corr += np.multiply(signal[t - lag, :], signal[t, :])
    return corr

def distribution_entropy(win):
    mut_inf = mutual_information(win)
    return np.sum([p_ij * np.log(p_ij) for p_ij in mut_inf / np.sum(mut_inf)])

def network_entropy(win):
    mutual_information_2d = -0.5 * (1 - (np.corrcoef(win.T, np.roll(win.T, -time_step_lag)))**2)
    np.fill_diagonal(mutual_information_2d, 0)
    s = np.repeat(np.sum(mutual_information_2d, axis = 0), win.shape[1] - 1)
    return flatten_remove_diag(mutual_information_2d) / s
    
def mutual_information(win):
    mutual_information = flatten_remove_diag(-0.5 * (1 - (np.corrcoef(win.T, np.roll(win.T, -time_step_lag)))**2))
    return mutual_information

config.add_record_object("electrode_sd", lambda win: np.std(win, axis = 0))
config.add_record_object('electrode_corr', lambda win: [[0]] * (n_channels * (n_channels - 1) // 2) if win.shape[0] < 2 else np.corrcoef(win.T)[np.triu_indices(n_channels, k = 1)])
config.add_record_object("autocorrelate", lambda win: calculate_auto_corr(win, 50))

time_step_lag = 15
config.add_record_object("mutual_information", lambda win: mutual_information(win))
config.add_record_object("distribution_entropy", lambda win: distribution_entropy(win))
config.add_record_object("network_entropy", lambda win: network_entropy(win))

recorder = FeatureRecorder(config)

In [13]:
recorder = feature_extractor.do_extraction(data.get_data().T, recorder, time_limit=100)

In [14]:
def build_visuality_graph(signal):
    n_graphs = signal.shape[1]
    T = signal.shape[0]    

In [22]:
remove_diag(np.array([[1,2 ,3 ], [4, 5,6 ], [7, 8, 9]])).ravel()

array([2, 3, 4, 6, 7, 8])